[View in Colaboratory](https://colab.research.google.com/github/rutgerhofste/notebook_gis_plotting/blob/master/05_folium_style_and_popup.ipynb)

# Visualize a geodataframe with popups and style



In [0]:
!pip install folium branca geojson geopandas --quiet

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored


In [0]:
import folium
import branca
import geojson
import pandas as pd
import geopandas as gpd

In [0]:
# load a sample geodataframe
gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [0]:
gdf_temp = gdf.copy()

In [0]:
# Create a column specifying the style
max_color = gdf["pop_est"].max()
min_color = gdf["pop_est"].min()
colormap = branca.colormap.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=3, vmax=max_color/2
)
colormap.caption = 'A colormap caption'
gdf_temp["style"] = gdf_temp.apply(lambda x: {'fillColor': colormap(x["pop_est"]), 'weight': 2, 'color': 'black'},axis=1)


In [0]:
def row_geojson_feature(row):    
    properties = row.drop("geometry").to_dict()
    geojson_feature = geojson.Feature(geometry=row["geometry"], properties=properties)
    return geojson_feature
  
def row_popup(row):
    width, height = 200,200
    attributes = pd.DataFrame(row.drop("geometry"))
    html_table = attributes.to_html()
    i_frame = branca.element.IFrame(html_table, width=width, height=height)
    return i_frame
 
def zip_features_popups(row):
    folium_feature = folium.features.GeoJson(row["gjson"])
    folium_feature.add_child(folium.Popup(row["i_frame"]))
    return folium_feature

In [0]:
# Generate geosjon representations of geometries
gdf_temp["gjson"] = gdf_temp.apply(row_geojson_feature,axis=1)

In [0]:
gdf_temp.iloc[0]["gjson"]

{"geometry": {"coordinates": [[[61.210817091725744, 35.650072333309225], [62.230651483005886, 35.270663967422294], [62.98466230657661, 35.40404083916762], [63.19353844590035, 35.857165635718914], [63.98289594915871, 36.0079574651466], [64.5464791197339, 36.31207326918427], [64.7461051776774, 37.111817735333304], [65.58894778835784, 37.30521678318564], [65.74563073106683, 37.66116404881207], [66.21738488145934, 37.39379018813392], [66.51860680528867, 37.36278432875879], [67.07578209825962, 37.35614390720929], [67.82999962755952, 37.144994004864685], [68.13556237170138, 37.02311513930431], [68.85944583524594, 37.344335842430596], [69.19627282092438, 37.15114350030743], [69.51878543485796, 37.60899669041342], [70.11657840361033, 37.58822276463209], [70.27057417184014, 37.735164699854025], [70.3763041523093, 38.13839590102752], [70.80682050973289, 38.486281643216415], [71.34813113799026, 38.25890534113216], [71.23940392444817, 37.953265082341886], [71.54191775908478, 37.905774441065645], [

In [0]:
# Create iframes as popups. 
gdf_temp["i_frame"] = gdf.apply(row_popup,1)

In [0]:
# Zip geosjon features and popups in folium features (This step is very slow, 
# todo: look into speed ups.)
gdf_temp["features"] = gdf_temp.apply(zip_features_popups,axis=1)

In [0]:
# Create folium feature group
feature_group_1 = folium.FeatureGroup(name="countries",control=True)
features_list = gdf_temp["features"].tolist()
for feature in features_list:
    feature.add_to(feature_group_1)

m2 = folium.Map(location = [52,4],
               zoom_start=6)
m2.add_child(feature_group_1)
m2.add_child(folium.map.LayerControl())
m2.add_child(colormap)
m2